# DeepSeek Agent 实战：小红书爆款文案生成助手

本 Notebook 将指导您如何使用 DeepSeek LLM 构建一个能够生成小红书爆款文案的智能 Agent。我们将从需求拆解开始，逐步定义 Agent 的系统提示词 (System Prompt)、外部工具 (Tools)，并实现其核心的工作流程，最终生成符合小红书平台特点的文案。

## 1. 环境准备与DeepSeek API配置

In [1]:
# 安装所需库
!pip install langchain langchain-openai langchain-community chromadb beautifulsoup4 markdown

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [2]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

## 知识库构建
此模块负责读取您的 Markdown 文件，将其分割成块，使用 bge-m3 模型进行嵌入，并构建一个 Chroma 向量数据库。

In [3]:
# 第1步：导入库，获取硅基流动api-key，为调用嵌入模型准备
import os
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 SILICONFLOW_API_KEY
api_key = os.getenv("SILICONFLOW_API_KEY")
if not api_key:
    raise ValueError("请设置 SILICONFLOW_API_KEY 环境变量")

In [4]:
# 第2步：加载和分割文档
# 获取知识文件路径
import pathlib
# 获取当前Jupyter Notebook的工作目录
# 在标准Jupyter环境中，.cwd() 通常就是.ipynb文件所在的目录
current_dir = pathlib.Path.cwd()

# 使用 / 操作符来构建路径，非常直观和优雅
# 这会从当前目录出发，寻找名为'docs'的文件夹，再寻找里面的文件
file_path = current_dir / "docs" / "skincare_products.md"

# 我们可以打印一下，看看它解析出的完整路径是什么
print(f"当前工作目录: {current_dir}")
print(f"构建的文件路径: {file_path}")
print(f"文件是否存在: {file_path.exists()}") # 检查路径是否正确

# 加载Markdown文件
loader = TextLoader(file_path, encoding="utf-8")
docs = loader.load()

# 定义Markdown的切分规则
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# 使用Markdown标题进行分割
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(docs[0].page_content)

print(f"文档被成功分割成 {len(md_header_splits)} 个块。")
# 打印第一个块作为示例
print("\n--- 第一个数据块示例 ---")
print(md_header_splits[0].page_content)
print("\n--- 它的元数据 ---")
print(md_header_splits[0].metadata)

当前工作目录: D:\AgentsDEV\Homework\deepseek-quickstart\rednote
构建的文件路径: D:\AgentsDEV\Homework\deepseek-quickstart\rednote\docs\skincare_products.md
文件是否存在: True
文档被成功分割成 7 个块。

--- 第一个数据块示例 ---
-   **产品口号**: 一片解锁水光肌，告别沙漠干皮
-   **核心成分**:
-   **深海蓝藻提取物**: 源自纯净深海，富含天然多糖和矿物质，具备卓越的锁水和保湿能力。
-   **小分子透明质酸**: 渗透至肌肤底层，快速补充水分，令肌肤饱满弹润。
-   **积雪草提取物**: 经典的舒缓修护成分，能有效缓解因干燥引起的泛红和不适。
-   **主要功效**:
-   **深层补水**: 瞬间为肌肤注入大量水分，长效保湿。
-   **修护屏障**: 强化肌肤锁水屏障，减少水分流失。
-   **舒缓敏感**: 镇静肌肤，改善干燥、泛红等敏感问题。
-   **适合肤质**: 所有肤质，尤其推荐干燥、缺水及敏感性肌肤。
-   **使用方法**: 洁面后，取出面膜并展开，敷于面部并调整至贴合。静待15-20分钟后取下，轻轻按摩面部，帮助剩余精华液吸收。
-   **产品规格**: 25ml/片, 5片/盒  
---

--- 它的元数据 ---
{'Header 1': '护肤品产品说明书知识库', 'Header 2': '面膜系列', 'Header 3': '深海蓝藻高保湿面膜 (Deep Sea Blue Algae Hydrating Mask)'}


In [20]:
# 第3步:初始化嵌入模型并创建chroma向量库
# 初始化硅基流动的 BAAI/bge-large-zh-v1.5 嵌入模型
# LangChain的OpenAIEmbeddings类可以通过配置base_url来兼容大部分OpenAI API标准的模型服务
import shutil  # Python标准库，提供了许多高级的文件和目录操作功能。

embedding_model = OpenAIEmbeddings(
    model="BAAI/bge-large-zh-v1.5",
    api_key=os.getenv("SILICONFLOW_API_KEY"),
    base_url="https://api.siliconflow.cn/v1"
)

# 定义向量数据库的持久化路径
persist_directory = './chroma_db_skincare'

# 检查向量数据库目录是否已经存在
if os.path.exists(persist_directory):
    print(f"检测到已存在的向量数据库 '{persist_directory}'，正在删除...")
    # 使用 shutil.rmtree 来递归删除目录及其所有内容
    shutil.rmtree(persist_directory)
    print("旧数据库删除成功。")
# --------------------

# 创建并持久化向量数据库
# 这会将分割后的文档进行嵌入并存入ChromaDB
print("正在创建向量数据库，请稍候...")
vectordb = Chroma.from_documents(
    documents=md_header_splits,
    embedding=embedding_model,
    persist_directory=persist_directory
)
print(f"向量数据库创建成功并已保存至 '{persist_directory}' 文件夹！")

正在创建向量数据库，请稍候...
向量数据库创建成功并已保存至 './chroma_db_skincare' 文件夹！


## 知识库查询
此模块加载已创建的向量库，并构建一个检索器（Retriever），用于查询相关信息

In [6]:
# 加载向量库并创建检索器
# 加载持久化的向量数据库
# 注意：必须使用与创建时相同的 embedding_model
print("正在从本地加载向量数据库...")
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model
)

# 创建一个检索器，并设置它只返回最相关的1个结果
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

print("检索器已准备就绪！")

# --- 功能验证测试 ---
query = "哪款产品适合熬夜党用来修护黑眼圈？"
search_results = retriever.invoke(query)

print(f"\n--- 对查询 '{query}' 的检索结果 ---")
# 结果是一个Document对象的列表
print(search_results[0].page_content)
print(f"\n来源元数据: {search_results[0].metadata}")

正在从本地加载向量数据库...
检索器已准备就绪！


C:\Users\zf\AppData\Local\Temp\ipykernel_32408\644406854.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(



--- 对查询 '哪款产品适合熬夜党用来修护黑眼圈？' 的检索结果 ---
-   **产品口号**: 云朵般泡沫，给肌肤最温柔的SPA
-   **核心成分**:
-   **椰油酰甘氨酸钠**: 源自天然的氨基酸表面活性剂，亲肤性极佳，清洁力适中。
-   **神经酰胺**: 保护并修护皮脂膜，维持肌肤水润，洗后不紧绷。
-   **甘油**: 基础保湿成分，在清洁过程中锁住肌肤水分。
-   **主要功效**:
-   **温和清洁**: 有效去除日常污垢和多余油脂，不伤害肌肤屏障。
-   **水润保湿**: 独特的配方能在洁面后维持肌肤的水润感。
-   **平衡水油**: 调节肌肤至健康的中性状态。
-   **适合肤质**: 所有肤质，包括干性、油性及最娇嫩的敏感性肌肤。
-   **使用方法**: 湿润面部后，取适量（约2cm）于掌心，加水揉搓出丰富泡沫。将泡沫在面部轻轻打圈按摩，然后用清水彻底冲洗干净。
-   **产品规格**: 100g/支

来源元数据: {'Header 1': '护肤品产品说明书知识库', 'Header 3': '氨基酸净透洁面乳 (Amino Acid Gentle Cleanser)', 'Header 2': '洁面与调理系列'}


## 构建最终的查询函数
现在，我们将上述检索功能封装成您要求的 query_product_database 函数。

In [7]:
# 定义 query_product_database 函数
def query_product_database(product_name: str) -> str:
    """
    通过RAG从向量知识库中检索产品信息。
    
    参数:
    product_name (str): 需要查询的产品名称或相关描述。
    
    返回:
    str: 格式化的产品详细信息，如果找不到则返回提示信息。
    """
    print(f"[RAG Tool Call] 正在从向量库中检索关于 '{product_name}' 的信息...")
    
    # 使用之前创建的retriever进行查询
    results = retriever.invoke(product_name)
    
    if not results:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"
    
    # 提取最相关的文档内容和元数据
    top_result_doc = results[0]
    content = top_result_doc.page_content
    metadata = top_result_doc.metadata
    
    # 按照您希望的格式（保留原有格式），将标题和内容组合起来
    # 从元数据中获取产品名（Header 3）
    product_title = metadata.get("Header 3", product_name)
    
    # 组合成 Markdown 格式的字符串
    formatted_output = f"### {product_title}\n\n{content}"
    
    return formatted_output

# --- 最终函数功能验证 ---
product_to_query = "光感焕白精华"
product_info = query_product_database(product_to_query)

print("\n--- query_product_database 函数调用结果 ---")
print(product_info)

print("\n" + "="*50 + "\n")

# 测试另一个产品
product_to_query_2 = "氨基酸洗面奶怎么样" # 使用模糊查询
product_info_2 = query_product_database(product_to_query_2)
print("--- 模糊查询结果 ---")
print(product_info_2)

[RAG Tool Call] 正在从向量库中检索关于 '光感焕白精华' 的信息...

--- query_product_database 函数调用结果 ---
### 神经酰胺屏障修护面霜 (Ceramide Barrier Repair Cream)

-   **产品口号**: 重建肌肤“水泥墙”，锁住健康与水润
-   **核心成分**:
-   **三重神经酰胺 (NP, AP, EOP)**: 模拟皮脂膜结构，精准补充细胞间脂质，强韧肌肤屏障。
-   **角鲨烷**: 高亲肤性油脂，形成天然保护膜，提供持久滋润。
-   **牛油果树果脂**: 富含维生素和脂肪酸，深度滋养，缓解干燥起皮。
-   **主要功效**:
-   **强韧屏障**: 修复受损的肌肤物理屏障，提升抵御力。
-   **长效保湿**: 深度滋润，持久锁水，缓解干燥紧绷。
-   **维稳修护**: 改善换季、刷酸等引起的泛红、脱皮等问题。
-   **适合肤质**: 干性、极干性肌肤及屏障受损的敏感性肌肤。
-   **使用方法**: 在护肤的最后一步，取适量面霜于掌心乳化后，均匀按压于面部和颈部，直至完全吸收。
-   **产品规格**: 50g/罐


[RAG Tool Call] 正在从向量库中检索关于 '氨基酸洗面奶怎么样' 的信息...
--- 模糊查询结果 ---
### 深海蓝藻高保湿面膜 (Deep Sea Blue Algae Hydrating Mask)

-   **产品口号**: 一片解锁水光肌，告别沙漠干皮
-   **核心成分**:
-   **深海蓝藻提取物**: 源自纯净深海，富含天然多糖和矿物质，具备卓越的锁水和保湿能力。
-   **小分子透明质酸**: 渗透至肌肤底层，快速补充水分，令肌肤饱满弹润。
-   **积雪草提取物**: 经典的舒缓修护成分，能有效缓解因干燥引起的泛红和不适。
-   **主要功效**:
-   **深层补水**: 瞬间为肌肤注入大量水分，长效保湿。
-   **修护屏障**: 强化肌肤锁水屏障，减少水分流失。
-   **舒缓敏感**: 镇静肌肤，改善干燥、泛红等敏感问题。
-   **适合肤质**: 所有肤质，尤其推荐干燥、缺水及敏感性肌肤。
-   **使用方法**: 洁面后，取出面膜并展开，敷于面部

## 2. 需求拆解与Agent任务规划

#### 用户痛点与核心需求：
*   **效率低下：** 人工创作周期长，难以满足高频发布需求。
*   **创意瓶颈：** 难以持续产出新颖、吸引人的爆款创意。
*   **趋势捕捉难：** 实时流行元素难以快速融入文案。
*   **平台特性把握：** 小红书特有风格（标题、正文、标签、表情）难以精准复制。

#### “爆款”文案的特征：
*   **强吸引力标题：** 制造好奇、痛点共鸣、利益点清晰。
*   **沉浸式正文：** 真实体验分享、细节描述、情感共鸣。
*   **精准且多样标签：** 热门话题、品牌词、产品词、垂直领域词。
*   **生动表情符号：** 增强表达力，提升活泼感。
*   **清晰的行动召唤 (CTA)。**

#### Agent 任务规划：核心工作流
1.  **用户指令接收：** 接收产品信息、主题、风格等。
2.  **信息收集 (Web Search/DB Query)：** 实时搜索行业趋势、热门话题、竞品分析、产品卖点。
3.  **内容构思与初稿生成 (LLM)：** 结合所有信息，撰写标题、正文、标签、表情符号。
4.  **风格与格式优化 (LLM)：** 根据小红书平台特点和指定风格，对文案进行润色和结构调整。
5.  **最终输出：** 呈现完整文案。

## 3. 爆款文案生成逻辑与 Prompt 设计

### 3.1 System Prompt (系统提示词)

System Prompt 是 Agent 的“大脑”和“行为准则”。它定义了 Agent 的角色、目标以及工作方式。我们将采用 `Thought-Action-Observation` (ReAct) 模式来引导 DeepSeek 的推理过程。

In [8]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

### 3.2 Tools (工具定义)

Agent 的“双手”由一系列可调用的工具组成。这些工具扩展了 LLM 的能力，使其能够获取实时信息、查询数据库或执行特定操作。在这里，我们定义了三个核心工具：

*   `search_web`: 用于搜索互联网上的实时信息，如最新趋势、用户评价等。
*   `query_product_database`: 用于查询产品数据库，获取产品的详细卖点和特点。**此工具为模拟**。
*   `generate_emoji`: 用于根据文案内容生成恰当的表情符号。**此工具为模拟**。

In [9]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

### 3.3 模拟工具实现

由于我们无法直接调用真实的外部 API (如Google Search或内部产品数据库)，我们将创建一些模拟 (Mock) 工具函数来演示 Agent 的工作流程。在实际应用中，您需要将这些模拟函数替换为真实的 API 调用。

In [10]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "深海蓝藻保湿面膜" in product_name:
        return "深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。"
    elif "美白精华" in product_name:
        return "美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。规格：30ml。"
    elif "氨基酸净透洁面乳" in product_name:
        return "氨基酸净透洁面乳：采用温和的氨基酸表面活性剂，泡沫绵密，能有效清洁面部多余油脂和污垢，同时保护皮脂膜，洗后不紧绷。适合所有肤质，包括敏感肌。规格：100g。"
    elif "酵母修护眼霜" in product_name:
        return "酵母修护眼霜：核心成分为二裂酵母发酵产物溶胞物，能有效淡化黑眼圈和眼部细纹，提拉紧致眼周肌肤。质地滋润不油腻，易于吸收。规格：15g。"
    elif "果酸焕肤精华水" in product_name:
        return "果酸焕肤精华水：含低浓度复合果酸（AHA），能温和剥落老废角质，疏通毛孔，改善闭口粉刺，使肌肤更加平滑透亮。建议夜间使用，并注意日间防晒。规格：150ml。"
    elif "金盏花舒缓爽肤水" in product_name:
        return "金盏花舒缓爽肤水：蕴含天然金盏花菁萃，不含酒精，能有效舒缓肌肤泛红、平衡水油，调理痘痘肌。湿敷效果更佳，是油性及混合性肌肤的理想选择。规格：250ml。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))
        
# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
   # "query_product_database": mock_query_product_database,
    "query_product_database": query_product_database, # 映射为真实的函数
    "generate_emoji": mock_generate_emoji,
}

## 4. 实战：构建小红书文案生成 Agent

现在，我们将把 System Prompt、工具定义和模拟工具函数整合起来，构建出能够自动执行的 DeepSeek Agent 工作流。核心是 `generate_rednote` 函数，它通过一个循环来模拟 Agent 的 `Thought-Action-Observation` 过程。

In [11]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]
    
    iteration_count = 0
    final_response = None
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION, # 告知模型可用的工具
                tool_choice="auto" # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message
            
            # **ReAct模式：处理工具调用**
            if response_message.tool_calls: # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message) # 将工具调用信息添加到对话历史
                
                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")
                    
                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result) # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs) # 将工具执行结果作为 Observation 添加到对话历史
                
            # **ReAct 模式：处理最终内容**
            elif response_message.content: # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")
                
                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
                
                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break
                
        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break
    
    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

## 5. 实际测试与文案生成

现在，让我们调用我们构建的 `generate_rednote` 函数，看看它能生成什么样的爆款文案！

In [12]:
# 测试案例 1: 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '深海蓝藻保湿面膜'}
[RAG Tool Call] 正在从向量库中检索关于 '深海蓝藻保湿面膜' 的信息...
Observation: 工具返回结果：### 神经酰胺屏障修护面霜 (Ceramide Barrier Repair Cream)

-   **产品口号**: 重建肌肤“水泥墙”，锁住健康与水润
-   **核心成分**:
-   **三重神经酰胺 (NP, AP, EOP)**: 模拟皮脂膜结构，精准补充细胞间脂质，强韧肌肤屏障。
-   **角鲨烷**: 高亲肤性油脂，形成天然保护膜，提供持久滋润。
-   **牛油果树果脂**: 富含维生素和脂肪酸，深度滋养，缓解干燥起皮。
-   **主要功效**:
-   **强韧屏障**: 修复受损的肌肤物理屏障，提升抵御力。
-   **长效保湿**: 深度滋润，持久锁水，缓解干燥紧绷。
-   **维稳修护**: 改善换季、刷酸等引起的泛红、脱皮等问题。
-   **适合肤质**: 干性、极干性肌肤及屏障受损的敏感性肌肤。
-   **使用方法**: 在护肤的最后一步，取适量面霜于掌心乳化后，均匀按压于面部和颈部，直至完全吸收。
-   **产品规格**: 50g/罐
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '补水保湿、惊喜效果'}
[Tool Call] 模拟生成表情符号，上下文：补水保湿、惊喜效果
Observation: 工具返回结果：['💦', '💧', '🌊', '✨']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "💦深海蓝藻保湿面膜｜干皮救星！用完皮肤水嫩到发光✨",
  "body": "姐妹们！发现了一款神仙面膜——深海蓝藻保湿面膜！简直是干皮的救星啊！🌊\n\n🌟核心成分：\n- 深海蓝藻精华：深层补水，锁住水分不流失！\n- 三重神

In [13]:
# 测试案例 2: 美白精华
product_name_2 = "美白精华"
tone_style_2 = "知性温柔"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '美白精华'}
[RAG Tool Call] 正在从向量库中检索关于 '美白精华' 的信息...
Observation: 工具返回结果：### 神经酰胺屏障修护面霜 (Ceramide Barrier Repair Cream)

-   **产品口号**: 重建肌肤“水泥墙”，锁住健康与水润
-   **核心成分**:
-   **三重神经酰胺 (NP, AP, EOP)**: 模拟皮脂膜结构，精准补充细胞间脂质，强韧肌肤屏障。
-   **角鲨烷**: 高亲肤性油脂，形成天然保护膜，提供持久滋润。
-   **牛油果树果脂**: 富含维生素和脂肪酸，深度滋养，缓解干燥起皮。
-   **主要功效**:
-   **强韧屏障**: 修复受损的肌肤物理屏障，提升抵御力。
-   **长效保湿**: 深度滋润，持久锁水，缓解干燥紧绷。
-   **维稳修护**: 改善换季、刷酸等引起的泛红、脱皮等问题。
-   **适合肤质**: 干性、极干性肌肤及屏障受损的敏感性肌肤。
-   **使用方法**: 在护肤的最后一步，取适量面霜于掌心乳化后，均匀按压于面部和颈部，直至完全吸收。
-   **产品规格**: 50g/罐
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '美白精华'}
[Tool Call] 模拟生成表情符号，上下文：美白精华
Observation: 工具返回结果：['💧']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉 ✨",
  "body": "姐妹们！我终于找到了让我从黄皮逆袭冷白皮的神仙精华！💖\n\n作为一个常年熬夜的社畜，皮肤暗沉、肤色不均简直是我的噩梦 😭 直到闺蜜推荐了这款美白精华，我才知道什么叫“一白遮百丑”！\n\n🌟 **核心成分**：\n- 高浓度

In [14]:
# 测试案例 3: 氨基酸净透洁面乳
product_name_3 = "氨基酸净透洁面乳"
tone_style_3 = "专业测评"
result_3 = generate_rednote(product_name_3, tone_style_3)

print("\n--- 生成的文案 3 ---")
print(result_3)


🚀 启动小红书文案生成助手，产品：氨基酸净透洁面乳，风格：专业测评

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '氨基酸净透洁面乳'}
[RAG Tool Call] 正在从向量库中检索关于 '氨基酸净透洁面乳' 的信息...
Observation: 工具返回结果：### 氨基酸净透洁面乳 (Amino Acid Gentle Cleanser)

-   **产品口号**: 云朵般泡沫，给肌肤最温柔的SPA
-   **核心成分**:
-   **椰油酰甘氨酸钠**: 源自天然的氨基酸表面活性剂，亲肤性极佳，清洁力适中。
-   **神经酰胺**: 保护并修护皮脂膜，维持肌肤水润，洗后不紧绷。
-   **甘油**: 基础保湿成分，在清洁过程中锁住肌肤水分。
-   **主要功效**:
-   **温和清洁**: 有效去除日常污垢和多余油脂，不伤害肌肤屏障。
-   **水润保湿**: 独特的配方能在洁面后维持肌肤的水润感。
-   **平衡水油**: 调节肌肤至健康的中性状态。
-   **适合肤质**: 所有肤质，包括干性、油性及最娇嫩的敏感性肌肤。
-   **使用方法**: 湿润面部后，取适量（约2cm）于掌心，加水揉搓出丰富泡沫。将泡沫在面部轻轻打圈按摩，然后用清水彻底冲洗干净。
-   **产品规格**: 100g/支
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '温和清洁、水润保湿、专业测评'}
[Tool Call] 模拟生成表情符号，上下文：温和清洁、水润保湿、专业测评
Observation: 工具返回结果：['💦', '💧', '🌊', '✨']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "✨氨基酸净透洁面乳测评｜云朵般泡沫，洗出婴儿肌！💦",
  "body": "姐妹们！今天来测评一款我最近挖到的宝藏洁面乳——「氨基酸净透洁面乳」！🌟\n\n💧 **核心成分**：\n- 椰油酰甘氨酸钠：天然氨基酸表活，

In [15]:
# 测试案例 4: 酵母修护眼霜
product_name_4 = "酵母修护眼霜"
tone_style_4 = "姐妹分享"
result_4 = generate_rednote(product_name_4, tone_style_4)

print("\n--- 生成的文案 4 ---")
print(result_4)



🚀 启动小红书文案生成助手，产品：酵母修护眼霜，风格：姐妹分享

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '酵母修护眼霜'}
[RAG Tool Call] 正在从向量库中检索关于 '酵母修护眼霜' 的信息...
Observation: 工具返回结果：### 氨基酸净透洁面乳 (Amino Acid Gentle Cleanser)

-   **产品口号**: 云朵般泡沫，给肌肤最温柔的SPA
-   **核心成分**:
-   **椰油酰甘氨酸钠**: 源自天然的氨基酸表面活性剂，亲肤性极佳，清洁力适中。
-   **神经酰胺**: 保护并修护皮脂膜，维持肌肤水润，洗后不紧绷。
-   **甘油**: 基础保湿成分，在清洁过程中锁住肌肤水分。
-   **主要功效**:
-   **温和清洁**: 有效去除日常污垢和多余油脂，不伤害肌肤屏障。
-   **水润保湿**: 独特的配方能在洁面后维持肌肤的水润感。
-   **平衡水油**: 调节肌肤至健康的中性状态。
-   **适合肤质**: 所有肤质，包括干性、油性及最娇嫩的敏感性肌肤。
-   **使用方法**: 湿润面部后，取适量（约2cm）于掌心，加水揉搓出丰富泡沫。将泡沫在面部轻轻打圈按摩，然后用清水彻底冲洗干净。
-   **产品规格**: 100g/支
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '酵母修护眼霜，温和修护，淡化细纹'}
[Tool Call] 模拟生成表情符号，上下文：酵母修护眼霜，温和修护，淡化细纹
Observation: 工具返回结果：['🤩']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "🤩酵母修护眼霜实测！熬夜党必备，黑眼圈细纹统统说拜拜～",
  "body": "姐妹们！最近发现了一款神仙眼霜——酵母修护眼霜！作为一个常年熬夜的社畜，黑眼圈和细纹简直是我的噩梦😭 但用了这款眼霜后，效果真的惊艳到我了！\n\n✨ **核心成分**：\

In [16]:
# 测试案例 5: 果酸焕肤精华水
product_name_5 = "果酸焕肤精华水"
tone_style_5 = "知识科普"
result_5 = generate_rednote(product_name_5, tone_style_5)

print("\n--- 生成的文案 5 ---")
print(result_5)


🚀 启动小红书文案生成助手，产品：果酸焕肤精华水，风格：知识科普

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '果酸焕肤精华水'}
[RAG Tool Call] 正在从向量库中检索关于 '果酸焕肤精华水' 的信息...
Observation: 工具返回结果：### 氨基酸净透洁面乳 (Amino Acid Gentle Cleanser)

-   **产品口号**: 云朵般泡沫，给肌肤最温柔的SPA
-   **核心成分**:
-   **椰油酰甘氨酸钠**: 源自天然的氨基酸表面活性剂，亲肤性极佳，清洁力适中。
-   **神经酰胺**: 保护并修护皮脂膜，维持肌肤水润，洗后不紧绷。
-   **甘油**: 基础保湿成分，在清洁过程中锁住肌肤水分。
-   **主要功效**:
-   **温和清洁**: 有效去除日常污垢和多余油脂，不伤害肌肤屏障。
-   **水润保湿**: 独特的配方能在洁面后维持肌肤的水润感。
-   **平衡水油**: 调节肌肤至健康的中性状态。
-   **适合肤质**: 所有肤质，包括干性、油性及最娇嫩的敏感性肌肤。
-   **使用方法**: 湿润面部后，取适量（约2cm）于掌心，加水揉搓出丰富泡沫。将泡沫在面部轻轻打圈按摩，然后用清水彻底冲洗干净。
-   **产品规格**: 100g/支
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '果酸焕肤精华水的知识科普与使用效果'}
[Tool Call] 模拟生成表情符号，上下文：果酸焕肤精华水的知识科普与使用效果
Observation: 工具返回结果：['✨']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "✨果酸焕肤精华水｜科学焕肤指南，告别闭口痘痘！✨",
  "body": "最近被这款「果酸焕肤精华水」圈粉了！作为一个成分党，我必须来科普一下它的神奇之处～\n\n🌟 **核心成分**：\n- **果酸（AHA）**：温和剥脱老废角质，促进肌

### 格式化 小红书文案

**格式化函数 `format_rednote_for_markdown` 的功能：**

1. 解析 JSON 字符串。
2. 提取标题、正文、标签和表情符号。
3. 将它们组合成一个易读的 Markdown 格式的文本。


**工作方式：**

1. **解析 JSON**：使用 `json.loads()` 将输入的字符串转换为 Python 字典。如果解析失败，会返回一个错误信息。
2. **提取数据**：使用 `.get()` 方法从字典中安全地提取 `title`、`body` 和 `hashtags`。使用 `.get()` 的好处是，如果某个键不存在，它会返回一个默认值（例如 `None` 或空列表），而不是抛出 `KeyError`。
3. **构建 Markdown 标题**：将 `title` 格式化为 Markdown 的二级标题 (`## Title`)。
4. **处理正文**：直接使用 `body`。由于小红书正文中的换行很重要，我们保留它们。
5. **处理 Hashtags**：将 `hashtags` 列表中的每个标签用空格连接起来，形成一行。
6. **表情符号 (Emojis)**：在小红书的实际发布中，表情符号通常已经嵌入在标题和正文中了。这个函数没有单独列出它们，因为这通常不是最终发布格式的一部分。如果需要，可以取消注释相关代码来单独显示它们。
7. **返回结果**：返回拼接好的 Markdown 字符串，并使用 `.strip()` 去除可能存在于末尾的多余空白。

In [17]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [18]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟",
  "body": "姐妹们！我终于找到了我的本命美白精华！💖\\n\\n作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩\\n\\n🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！\\n💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！\\n🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！\\n\\n使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～\\n\\n真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤好物", "#冷白皮"],
  "emojis": ["✨", "💖", "🤩", "💧", "🌿"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)


--- 格式化后的小红书文案 (Markdown) ---
## ✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟

姐妹们！我终于找到了我的本命美白精华！💖

作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩

🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！
💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！
🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！

使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～

真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖

#美白精华 #提亮肤色 #淡化痘印 #护肤好物 #冷白皮

--- 格式化错误示例 ---
错误：无法解析 JSON 字符串 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
原始字符串：
{'title': 'Test', 'body': 'This is not valid json'}


In [19]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_1)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 💦深海蓝藻保湿面膜｜干皮救星！用完皮肤水嫩到发光✨

姐妹们！发现了一款神仙面膜——深海蓝藻保湿面膜！简直是干皮的救星啊！🌊

🌟核心成分：
- 深海蓝藻精华：深层补水，锁住水分不流失！
- 三重神经酰胺：修复肌肤屏障，告别干燥起皮！
- 角鲨烷+牛油果树果脂：滋养肌肤，让脸蛋嫩到掐出水！💧

💖使用感受：
敷上脸的那一刻，冰冰凉凉的超舒服！精华液超多，敷完皮肤立马水润透亮，连毛孔都细腻了！第二天上妆完全不卡粉，简直惊喜到尖叫！✨

📌适合人群：干皮、混干皮、屏障受损的敏感肌！

💡使用方法：洁面后敷15-20分钟，轻轻按摩至精华吸收，后续护肤效果翻倍！

#干皮救星 #深海蓝藻面膜 #保湿面膜 #护肤必备 #敏感肌友好

#干皮救星 #深海蓝藻面膜 #保湿面膜 #护肤必备 #敏感肌友好


## 6. 评估与优化

文案生成并非一蹴而就，需要持续的评估和优化。本节讨论一些评估方法和优化策略。

#### 评估文案质量：
*   **客观量化评估 (数据)：**
    *   **点赞/收藏/评论/分享：** 基础互动
    *   **曝光/阅读/点击/涨粉：：** 流量与曝光
    *   **停留时长/截图率：** 用户行为。
    *   **商品页浏览/加购/ROI/成交转化：** 商业价值
    *   **爆文率/同类横向对比：** 竞争对比
*   **主观内部评估 (人工)：**
    *   **相关性：** 是否符合产品特点和主题。
    *   **吸引力：** 标题是否抓人，内容是否流畅。
    *   **合规性：** 是否有敏感词、违规宣传。
    *   **风格匹配：** 是否符合小红书调性和指定语气。
    *   **用户画像：** 目标人群年龄、地域、兴趣标签。



#### 优化迭代方法：
*   **Prompt 调整：** 根据评估结果，精修 System Prompt、User Prompt，增加或修改 Few-shot 示例。
*   **工具扩充：** 引入新的工具（如敏感词检测工具、竞品分析工具）。
*   **RAG (检索增强生成)：** 结合更精准的内部知识库，减少幻觉。


## 7. 总结与展望

通过本次实战，我们成功构建了一个基于 DeepSeek Agent 的小红书爆款文案生成助手。我们学习了如何拆解需求、设计 Prompt、定义工具，并实现 Agent 的核心工作流。

Agent 在内容营销领域的潜力巨大，未来可以进一步拓展到：

*   **超个性化内容：** 根据用户数据，生成一对一的定制文案。
*   **多模态内容创作：** 结合图片、视频生成，实现图文音视频一体化。
*   **智能营销决策：** Agent 不仅生成内容，还能分析效果并给出投放建议。
*   **跨平台适配：** 快速生成适应不同社交媒体平台风格的文案。

同时，我们也需关注挑战，如确保内容真实性、处理高度主观情感、与现有工作流的无缝集成等。Agent 技术仍在快速发展，期待未来能带来更多惊喜！